# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 11:59AM,247268,10,CTF0011412-BO,"Citieffe, Inc.",Released
1,Sep 26 2022 11:58AM,247267,10,CTF0011365-BO,"Citieffe, Inc.",Released
2,Sep 26 2022 11:57AM,247265,16,SHL-8575437,"SHL Pharma, LLC",Released
3,Sep 26 2022 11:57AM,247266,10,PRF-37054,Bio-PRF,Released
4,Sep 26 2022 11:54AM,247264,19,8575432,"Graystone, LLC",Executing
5,Sep 26 2022 11:54AM,247264,19,8575435,"Graystone, LLC",Released
6,Sep 26 2022 11:49AM,247262,20,8445950-BO,"ACI Healthcare USA, Inc.",Released
7,Sep 26 2022 11:49AM,247262,20,8436837-BO,"ACI Healthcare USA, Inc.",Released
8,Sep 26 2022 11:49AM,247262,20,8461835-BO,"ACI Healthcare USA, Inc.",Released
9,Sep 26 2022 11:49AM,247262,20,8458019-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247264,Released,1
26,247265,Released,1
27,247266,Released,1
28,247267,Released,1
29,247268,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247264,1.0,1.0
247265,NaN,1.0
247266,NaN,1.0
247267,NaN,1.0
247268,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247197,1.0,0.0
247199,1.0,0.0
247219,0.0,15.0
247220,0.0,3.0
247223,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247197,1,0
247199,1,0
247219,0,15
247220,0,3
247223,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247197,1,0
1,247199,1,0
2,247219,0,15
3,247220,0,3
4,247223,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247197,1,
1,247199,1,
2,247219,,15
3,247220,,3
4,247223,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc."
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc."
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC"
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC"
6,Sep 26 2022 11:49AM,247262,20,"ACI Healthcare USA, Inc."
57,Sep 26 2022 11:44AM,247263,16,ACG North America LLC
58,Sep 26 2022 11:41AM,247260,10,ISDIN Corporation
64,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health
66,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",,1
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",,1
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC",,1
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF,,1
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC",1,1
5,Sep 26 2022 11:49AM,247262,20,"ACI Healthcare USA, Inc.",,51
6,Sep 26 2022 11:44AM,247263,16,ACG North America LLC,,1
7,Sep 26 2022 11:41AM,247260,10,ISDIN Corporation,,6
8,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,,2
9,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",1,
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",1,
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC",1,
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF,1,
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC",1,1
5,Sep 26 2022 11:49AM,247262,20,"ACI Healthcare USA, Inc.",51,
6,Sep 26 2022 11:44AM,247263,16,ACG North America LLC,1,
7,Sep 26 2022 11:41AM,247260,10,ISDIN Corporation,6,
8,Sep 26 2022 11:27AM,247259,10,Ivaoes Animal Health,2,
9,Sep 26 2022 11:23AM,247254,10,Eywa Pharma Inc.,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",1,
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",1,
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC",1,
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF,1,
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC",1,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",1.0,NaN
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",1.0,NaN
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC",1.0,NaN
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF,1.0,NaN
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC",1.0,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",1.0,0.0
1,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",1.0,0.0
2,Sep 26 2022 11:57AM,247265,16,"SHL Pharma, LLC",1.0,0.0
3,Sep 26 2022 11:57AM,247266,10,Bio-PRF,1.0,0.0
4,Sep 26 2022 11:54AM,247264,19,"Graystone, LLC",1.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3708690,156.0,2.0
15,741750,19.0,19.0
16,741766,3.0,0.0
19,494522,2.0,1.0
20,247262,51.0,0.0
21,741642,0.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3708690,156.0,2.0
1,15,741750,19.0,19.0
2,16,741766,3.0,0.0
3,19,494522,2.0,1.0
4,20,247262,51.0,0.0
5,21,741642,0.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,156.0,2.0
1,15,19.0,19.0
2,16,3.0,0.0
3,19,2.0,1.0
4,20,51.0,0.0
5,21,0.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,156.0
1,15,Released,19.0
2,16,Released,3.0
3,19,Released,2.0
4,20,Released,51.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,2.0,19.0,0.0,1.0,0.0,3.0
Released,156.0,19.0,3.0,2.0,51.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,2.0,19.0,0.0,1.0,0.0,3.0
1,Released,156.0,19.0,3.0,2.0,51.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,2.0,19.0,0.0,1.0,0.0,3.0
1,Released,156.0,19.0,3.0,2.0,51.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()